In [5]:
from google.colab import files
files.upload() # kaggle.jsonをアップロード
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [6]:
import pandas as pd
import numpy as np
from sklearn.ensemble import \
RandomForestClassifier as RandomForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
# 上位k個の特徴量を選択するクラス、
# 上位percentileパーセントの特徴量を選択するクラス
from sklearn.feature_selection import SelectKBest, \
 SelectPercentile
# カイ二乗検定、分散分析のモジュールをインポート
from sklearn.feature_selection import chi2, f_classif
# ピアソン積率相関係数のモジュールをインポート
from scipy.stats import pearsonr
# 分散閾値により特徴選択を行うクラスをインポート
from sklearn.feature_selection import VarianceThreshold

# ピアソン積率相関係数
def Pearson_corr_coeff(X, y):
    X = np.array(X)
    y = np.array(y)
    pr = [[], []]
    for i in range(X.shape[1]):
        x = X[:, i]
        scores = pearsonr(y, x)
        pr[0].append(scores[0])
        pr[1].append(scores[1])
    return pr

# データの準備
def prepare():
    !kaggle datasets download -d \
    uciml/mushroom-classification
    !unzip mushroom-classification.zip

# カテゴリ特徴量を数字に変換
def replace_to_digit( dff ):
    t = list(set(dff.values.tolist()))
    t.sort()
    vals = [t.index(v)+1 for v in dff.values]
    return vals

# 分散が0の特徴量があると、分散分析などの計算に
# 影響が出るため、あらかじめ削除しておく
def removeByVarianceThreshold(df, X_train, features):
    c_filter = VarianceThreshold(threshold=0)
    c_filter.fit(X_train)
    nc = []
    for i, c in enumerate(c_filter.get_support()):
        if c == True:
            nc.append(features[i])
    X_train = df.loc[:, nc].values
    features = nc
    return X_train, features

# 前処理(欠損値の除去、カテゴリ特徴量の数値への変換)
def preprocess():
    df = pd.read_csv('mushrooms.csv')
    df = df.replace('?', 'NaN')
    df = df.dropna(how = 'any') 
    df['class'] = df['class'].map(
                  {'p': 1, 'e': 0}).astype(int)
    features = []
    for c in df.columns.values:
      if c != 'class':
        features.append(c)
        df[c] = replace_to_digit(df[c])
    X_train = df.loc[:, features].values
    y_train = df.loc[:, ['class']].values.ravel()

    # 分散閾値以下の分散を持つ特徴量を削除する
    X_train, features = removeByVarianceThreshold(
                             df, X_train, features)
    return X_train, y_train, features

# 特徴選択(SelectKBest)
def select_feature(selector_type, \
                   n_features, X_train, y_train, features):
    selector = SelectKBest(
     score_func=selector_type, k=n_features)
    # n_features次元の特徴量に変換
    X_new = selector.fit_transform(X_train, y_train) 
    feature_scores = list(zip(selector.scores_, 
                          features))
    # 特徴スコア順にソート
    sorted_feature_scores = sorted(feature_scores,
                                   reverse=True)
    selected_f = []
    # 特徴スコアの高い順に表示
    for i, fs in enumerate(
                     sorted_feature_scores[:n_features]):
        if fs[0] > 0 and fs[0] != None:
            print('[%d]: %s\t%.2lf' % (i+1, fs[1], fs[0]) )
            selected_f.append(fs[1])
        s_f = list(map(lambda i: features[i], 
                filter(
                lambda i: features[i] in selected_f, range( \
                              len(features)))))
    return s_f

# 特徴選択(SelectPercentile)
def select_feature_percentile(selector_type, 
                              percentile, 
                              X_train, y_train, 
                              features):

    selector = SelectPercentile(
                        score_func=selector_type,
                        percentile=percentile)
    # n_features次元の特徴量に変換
    X_new = selector.fit_transform(X_train, y_train) 
    # 選択された特徴の数を取得
    sel_count = np.count_nonzero(
                     selector.get_support()==True) 
    feature_scores = list(
                      zip(selector.scores_, 
                      features))
    # 特徴スコア順にソート
    sorted_feature_scores = sorted(
                            feature_scores, 
                            reverse=True)
    selected_f = []
    # 特徴スコアの高い順に表示
    for i, fs in enumerate( sorted_feature_scores[:sel_count]):
        if fs[0] > 0 and fs[0] != None:
            print('[%d]: %s\t%.2lf' % (i+1, fs[1], fs[0]) )
            selected_f.append(fs[1])
    s_f = list(map(lambda i: features[i], 
                filter(lambda i: features[i] \
                   in selected_f, range(len(features)))))
    return s_f

def main():
    prepare()
    X_train, y_train, features = preprocess()
    target_names = ['edible', 'poisonous']
    print('[Original Features]\n%s' % '\n'.join(features))

    # 特徴選択手法
    selectors = {'chi2': chi2,
                  'ANOVA': f_classif,
                  'Pearson': Pearson_corr_coeff}
    # SelectKBestを使う場合(n_features個の特徴量を選択)
    n_features = 5
    X_tr, X_te, y_tr, y_te = \
         train_test_split(X_train, y_train, 
         train_size=0.7, random_state=0)
    df_sel = pd.DataFrame(X_tr, columns=features)
    df_sel_te = pd.DataFrame(X_te, columns=features)
    for selN, selector_type in selectors.items():
        print('\n-*-*-*- Select by %s -*-*-*-' % selN)
        s_f =select_feature(selector_type, 
            n_features, X_tr, y_tr, features)
        X_tr_sel = df_sel.loc[:, s_f].values
        X_te_sel = df_sel_te.loc[:, s_f].values
        rf = RandomForest(n_estimators=100, 
          max_depth=4, random_state=0)
        rf.fit(X_tr_sel, y_tr)
        y_pred = rf.predict(X_te_sel)
        print(classification_report(y_te, y_pred,
                    target_names=target_names, 
                    zero_division=1))

    # SelectPercentileを使う場合
    # 上位percentileパーセントの特徴量を選択
    percentile = 10
    for selN, selector_type in selectors.items():
        print('\n-*-*-*- Select by %s -*-*-*-' % selN)
        s_f =select_feature_percentile(selector_type, 
                        percentile, X_train, y_train, 
                        features )
        X_tr_sel = df_sel.loc[:, s_f].values
        X_te_sel = df_sel_te.loc[:, s_f].values
        rf = RandomForest(n_estimators=100, 
                          max_depth=4, random_state=0)
        rf.fit(X_tr, y_tr)
        y_pred = rf.predict(X_te)
        print(classification_report(y_te, y_pred,
                    target_names=target_names, 
                    zero_division=1))

if __name__ == '__main__':
    main()
    

  0% 0.00/34.2k [00:00<?, ?B/s]
100% 34.2k/34.2k [00:00<00:00, 32.9MB/s]
Archive:  mushroom-classification.zip
  inflating: mushrooms.csv           
[Original Features]
cap-shape
cap-surface
cap-color
bruises
odor
gill-attachment
gill-spacing
gill-size
gill-color
stalk-shape
stalk-root
stalk-surface-above-ring
stalk-surface-below-ring
stalk-color-above-ring
stalk-color-below-ring
veil-color
ring-number
ring-type
spore-print-color
population
habitat

-*-*-*- Select by chi2 -*-*-*-
[1]: gill-color	3504.00
[2]: ring-type	897.99
[3]: stalk-root	430.39
[4]: habitat	295.17
[5]: gill-size	276.53
              precision    recall  f1-score   support

      edible       0.96      0.96      0.96      1272
   poisonous       0.96      0.96      0.96      1166

    accuracy                           0.96      2438
   macro avg       0.96      0.96      0.96      2438
weighted avg       0.96      0.96      0.96      2438


-*-*-*- Select by ANOVA -*-*-*-
[1]: gill-size	2399.59
[2]: gill-color	2264.